<a href="https://colab.research.google.com/github/jonnes-nascimento/machine_learning/blob/master/Introdu%C3%A7%C3%A3o_aos_algoritmos_de_linguagem_neural_NLP_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importando a base de dados do IMDB

In [5]:
from google.colab import drive
import pandas as pd

# monta a pasta do Google Drive que contém o arquivo com os dados de análise
drive.mount("/content/drive")

resenha = pd.read_csv("/content/drive/My Drive/Colab Notebooks/imdb-reviews-pt-br.csv")

resenha

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,id,text_en,text_pt,sentiment
0,1,Once again Mr. Costner has dragged out a movie...,"Mais uma vez, o Sr. Costner arrumou um filme p...",neg
1,2,This is an example of why the majority of acti...,Este é um exemplo do motivo pelo qual a maiori...,neg
2,3,"First of all I hate those moronic rappers, who...","Primeiro de tudo eu odeio esses raps imbecis, ...",neg
3,4,Not even the Beatles could write songs everyon...,Nem mesmo os Beatles puderam escrever músicas ...,neg
4,5,Brass pictures movies is not a fitting word fo...,Filmes de fotos de latão não é uma palavra apr...,neg
...,...,...,...,...
49454,49456,"Seeing as the vote average was pretty low, and...","Como a média de votos era muito baixa, e o fat...",pos
49455,49457,"The plot had some wretched, unbelievable twist...",O enredo teve algumas reviravoltas infelizes e...,pos
49456,49458,I am amazed at how this movieand most others h...,Estou espantado com a forma como este filme e ...,pos
49457,49459,A Christmas Together actually came before my t...,A Christmas Together realmente veio antes do m...,pos


# Separando os dados da resenha para treino e teste

In [0]:
from sklearn.model_selection import train_test_split

treino, teste, classe_treino, classe_teste = train_test_split(resenha.text_pt, resenha.sentiment, random_state = 42)

# Treinamento e classificação

In [17]:
# analisando dois casos

print("Negativo\n")
print(resenha.text_pt[189])
print("\n")

print("Positivo\n")
print(resenha.text_pt[49002])
print("\n")

# É importante termos os dados balanceados. No nosso caso, por exemplo, temos que nos certificar de termos dados positivos e negativos balanceados
print(resenha.sentiment.value_counts())

Negativo

Este é sem dúvida o pior filme que eu já vi. E acredite em mim, eu vi muitos filmes. A reviravolta inacreditável que o filme faz - passando de um extremamente mau filme "Formas de vida alienígenas habitam a terra", com um filme que tenta espalhar um arquicristiano "O dia do julgamento está próximo, buscar Jesus ou queimar por toda a eternidade em as dívidas ardentes do inferno "mensagem - deixou-me atordoado depois de ter sido atormentado por 85 minutos. Até mesmo os cristãos religiosos devem se envergonhar ou ficar furiosos ao ver suas crenças postadas dessa maneira. Eu não sabia o que fazer comigo quando assisti a atuação horrível que poderia ter sido realizada por crianças de 7 anos de idade. Simplesmente repugnante. Eu não sou cristão nem muito religioso. Mas se eu estivesse, não teria mais medo do Inferno. Rich Christiano mostrou ser algo muito pior.


Positivo

Crescendo em Nova York no final dos anos 80 e início dos anos 90, posso dizer pessoalmente que este é um dos d

In [20]:
# padronizando as classes dos dados

classificacao = resenha["sentiment"].replace(["neg", "pos"], [0, 1])
resenha["classificacao"] = classificacao

resenha

,id,text_en,text_pt,sentiment,classificacao
0,1,Once again Mr. Costner has dragged out a movie...,"Mais uma vez, o Sr. Costner arrumou um filme p...",neg,0
1,2,This is an example of why the majority of acti...,Este é um exemplo do motivo pelo qual a maiori...,neg,0
2,3,"First of all I hate those moronic rappers, who...","Primeiro de tudo eu odeio esses raps imbecis, ...",neg,0
3,4,Not even the Beatles could write songs everyon...,Nem mesmo os Beatles puderam escrever músicas ...,neg,0
4,5,Brass pictures movies is not a fitting word fo...,Filmes de fotos de latão não é uma palavra apr...,neg,0
...,...,...,...,...,...
49454,49456,"Seeing as the vote average was pretty low, and...","Como a média de votos era muito baixa, e o fat...",pos,1
49455,49457,"The plot had some wretched, unbelievable twist...",O enredo teve algumas reviravoltas infelizes e...,pos,1
49456,49458,I am amazed at how this movieand most others h...,Estou espantado com a forma como este filme e ...,pos,1
49457,49459,A Christmas Together actually came before my t...,A Christmas Together realmente veio antes do m...,pos,1


In [43]:
from sklearn.feature_extraction.text import CountVectorizer

texto = ["Assisti a um filme ótimo", "Assisti a um filme ruim"]

vetorizar = CountVectorizer(lowercase=False)

bag_of_words = vetorizar.fit_transform(texto)

# exibe o vocabulario criado
print(vetorizar.get_feature_names())

# gerando um dataframe da matriz esparsa criada pelo CountVectorizer
matriz_esparsa = pd.DataFrame.sparse.from_spmatrix(data=bag_of_words, columns=vetorizar.get_feature_names())

matriz_esparsa

['Assisti', 'filme', 'ruim', 'um', 'ótimo']


,Assisti,filme,ruim,um,ótimo
0,1,1,0,1,1
1,1,1,1,1,0


In [40]:
# limitando a quantidade de features no bag of words (50 palavras)
vetorizar = CountVectorizer(lowercase=False, max_features=50)

bag_of_words = vetorizar.fit_transform(resenha.text_pt)

print(bag_of_words.shape)

(49459, 50)


In [41]:
from sklearn.linear_model import LogisticRegression

# separando (segregando) os dados
treino, teste, classe_treino, classe_teste = train_test_split(bag_of_words, 
                                                              resenha.classificacao, 
                                                              random_state = 42)

regressao_logistica = LogisticRegression()
regressao_logistica.fit(treino, classe_treino)

acuracia = regressao_logistica.score(teste, classe_teste)

print(acuracia)

0.6583097452486858


In [42]:
# criando uma funcao para se obter a acuracia dos dados de teste e de treino

def classificar_texto(texto, coluna_texto, coluna_classificacao):

  vetorizar = CountVectorizer(lowercase=False, max_features=50)
  bag_of_words = vetorizar.fit_transform(texto[coluna_texto])

  treino, teste, classe_treino, classe_teste = train_test_split(bag_of_words, 
                                                              texto[coluna_classificacao], 
                                                              random_state = 42)
  
  regressao_logistica = LogisticRegression()
  regressao_logistica.fit(treino, classe_treino)

  return regressao_logistica.score(teste, classe_teste)

print(classificar_texto(resenha, "text_pt", "classificacao"))


0.6583097452486858
